In [1]:
import os, re, sys, json, csv, string, math
import numpy as np
import pandas as pd
import importlib
from collections import Counter, defaultdict

In [2]:
import pickle as pkl

In [3]:
listofnovels = pd.read_csv('ListOfNovels.txt')

In [4]:
listofnovels

novel                                   author  \
0                 A Room With A View                            E. M. Forster   
1               The Age of Innocence                            Edith Wharton   
2   Alice's Adventures in Wonderland                            Lewis Carroll   
3               Anne Of Green Gables                     Lucy Maud Montgomery   
4                       Daisy Miller                              Henry James   
5                               Emma                              Jane Austen   
6                  A Handful Of Dust                             Evelyn Waugh   
7                        Howards End                            E. M. Forster   
8                      Night And Day                              NightAndDay   
9                   Northanger Abbey                              Jane Austen   
10                        Persuasion                              Jane Austen   
11               Pride And Prejudice                              Jane Austen   
12             Sense And Sensibility                              Jane Austen   
13              The Sign of the Four                   Sir Arthur Conan Doyle   
14                     The Awakening                              Kate Chopin   
15                       The Gambler  Fyodor Dostoevsky (Trans. C.J. Hogarth)   
16                 The Invisible Man                              H. G. Wells   
17          The Man Who Was Thursday                         G. K. Chesterton   
18   The Mysterious Affair At Styles                          Agatha Christie   
19        The Picture Of Dorian Gray                              Oscar Wilde   
20             The Sport Of The Gods                     Paul Laurence Dunbar   
21                The Sun Also Rises                         Ernest Hemingway   

                         folder  
0                ARoomWithAView  
1                AgeOfInnocence  
2             AliceInWonderland  
3             AnneOfGreenGables  
4                   DaisyMiller  
5                          Emma  
6                 HandfulOfDust  
7                    HowardsEnd  
8                           NaN  
9               NorthangerAbbey  
10                   Persuasion  
11            PrideAndPrejudice  
12          SenseAndSensibility  
13                   SignOfFour  
14                 TheAwakening  
15                   TheGambler  
16              TheInvisibleMan  
17         TheManWhoWasThursday  
18  TheMysteriousAffairAtStyles  
19       ThePictureOfDorianGray  
20            TheSportOfTheGods  
21              TheSunAlsoRises

In [5]:
def get_arr(s):
    if isinstance(s, str):
        return eval(s)
    elif isinstance(s, list):
        return s
    else:
        raise TypeError

In [6]:
def verify_spans(df, ntext):
    qcount = 0
    mcount = 0
    
    qtot = 0
    mtot = 0
    
    for _, row in df.iterrows():
        qtexts = get_arr(row['qTextArr'])
        qspans = get_arr(row['qSpan'])

        mtexts = get_arr(row['menTexts'])
        mspans = get_arr(row['menSpans'])

        for qt, qs, mts, mss in zip(qtexts, qspans, mtexts, mspans):
            qtot += 1
            et = ntext[qs[0]:qs[1]]
            assert et.strip() == qt.strip(), print("mismatch at span {} and text {}".format(qs, qt))
            qcount += 1
            
            for mt, ms in zip(mts, mss):
                mtot += 1
                emt = ntext[ms[0]:ms[1]]
                assert mt.strip() == emt.strip(), print("mismatch at span {} and text {}".format(ms, mt))
                mcount += 1
    
    print("Verified {}/{} quotes".format(qcount, qtot))
    print("Verified {}/{} mentions".format(mcount, mtot))
    

In [7]:
novels = []
with open('ListOfNovels.txt', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        novels.append(row[-1])

In [8]:
print(novels), len(novels)

['ARoomWithAView', 'AgeOfInnocence', 'AliceInWonderland', 'AnneOfGreenGables', 'DaisyMiller', 'Emma', 'HandfulOfDust', 'HowardsEnd', 'NightAndDay', 'NorthangerAbbey', 'Persuasion', 'PrideAndPrejudice', 'SenseAndSensibility', 'SignOfFour', 'TheAwakening', 'TheGambler', 'TheInvisibleMan', 'TheManWhoWasThursday', 'TheMysteriousAffairAtStyles', 'ThePictureOfDorianGray', 'TheSportOfTheGods', 'TheSunAlsoRises']


(None, 22)

In [9]:
for novel in novels:
    print(novel)

    qdf = pd.read_csv('data/'+novel+'/quotations.csv', index_col=0)
    charDict = pkl.load(open('data/'+novel+'/charDict.pkl', 'rb'))
    
    print("Number of characters: {}".format(len(charDict['id2names'])))
                             
    with open('data/'+novel+'/text.txt', 'r') as f:
        ntext = f.read().strip()
    
    verify_spans(qdf, ntext)
    print()

ARoomWithAView
Number of characters: 67
Verified 1989/1989 quotes
Verified 3079/3079 mentions

AgeOfInnocence
Number of characters: 55
Verified 1592/1592 quotes
Verified 2549/2549 mentions

AliceInWonderland
Number of characters: 51
Verified 1048/1048 quotes
Verified 683/683 mentions

AnneOfGreenGables
Number of characters: 115
Verified 1779/1779 quotes
Verified 5168/5168 mentions

DaisyMiller
Number of characters: 10
Verified 725/725 quotes
Verified 1021/1021 mentions

Emma
Number of characters: 18
Verified 2109/2109 quotes
Verified 6310/6310 mentions

HandfulOfDust
Number of characters: 106
Verified 2617/2617 quotes
Verified 3198/3198 mentions

HowardsEnd
Number of characters: 57
Verified 3111/3111 quotes
Verified 4344/4344 mentions

NightAndDay
Number of characters: 54
Verified 2795/2795 quotes
Verified 3568/3568 mentions

NorthangerAbbey
Number of characters: 20
Verified 1014/1014 quotes
Verified 2351/2351 mentions

Persuasion
Number of characters: 35
Verified 682/682 quotes
Verifi

In [10]:
qdf

qTextArr  \
qId                                                        
Q0     ['I know a girl in Strasbourg who can show us ...   
Q1     ["She's been there two years and knows everyth...   
Q2     ['Hell,', 'why go to Strasbourg? We could go u...   
Q3     ["For God's sake,", "why did you say that abou...   
Q4     ['No, why should I? If I know an American girl...   
...                                                  ...   
Q2605  ['Want to go for a ride?', 'Want to ride throu...   
Q2606  ['Right,', "I haven't seen Madrid. I should se...   
Q2607                              ["I'll finish this,"]   
Q2608  ['Oh, Jake,', 'we could have had such a damned...   
Q2609           ['Yes,', "Isn't it pretty to think so?"]   

                                      qSpan  \
qId                                           
Q0                           [[6412, 6465]]   
Q1                           [[6554, 6654]]   
Q2             [[6777, 6782], [6793, 6860]]   
Q3             [[7015, 7030], [7042, 7121]]   
Q4                           [[7125, 7226]]   
...                                     ...   
Q2605  [[363401, 363423], [363434, 363464]]   
Q2606  [[363468, 363474], [363489, 363532]]   
Q2607                    [[363536, 363553]]   
Q2608  [[364182, 364191], [364206, 364257]]   
Q2609  [[364392, 364396], [364407, 364435]]   

                                                   qText       speaker  \
qId                                                                      
Q0     I know a girl in Strasbourg who can show us th...   Jake Barnes   
Q1     She's been there two years and knows everythin...   Jake Barnes   
Q2     Hell, why go to Strasbourg? We could go up to ...   Jake Barnes   
Q3     For God's sake, why did you say that about tha...   Robert Cohn   
Q4     No, why should I? If I know an American girl t...   Jake Barnes   
...                                                  ...           ...   
Q2605  Want to go for a ride? Want to ride through th...   Jake Barnes   
Q2606  Right, I haven't seen Madrid. I should see Mad...  Brett Ashley   
Q2607                                  I'll finish this,   Jake Barnes   
Q2608  Oh, Jake, we could have had such a damned good...  Brett Ashley   
Q2609                  Yes, Isn't it pretty to think so?   Jake Barnes   

                              addressee      qType  \
qId                                                  
Q0     ['Frances Clyne', 'Robert Cohn']  Anaphoric   
Q1     ['Frances Clyne', 'Robert Cohn']  Anaphoric   
Q2     ['Frances Clyne', 'Robert Cohn']  Anaphoric   
Q3                      ['Jake Barnes']  Anaphoric   
Q4                      ['Robert Cohn']   Implicit   
...                                 ...        ...   
Q2605                  ['Brett Ashley']  Anaphoric   
Q2606                   ['Jake Barnes']   Explicit   
Q2607                  ['Brett Ashley']  Anaphoric   
Q2608                   ['Jake Barnes']   Explicit   
Q2609                  ['Brett Ashley']  Anaphoric   

                                        refExp  \
qId                                              
Q0                                      I said   
Q1     I thought it was accidental and went on   
Q2                                      I said   
Q3                                     he said   
Q4                                         NaN   
...                                        ...   
Q2605                                   I said   
Q2606                               Brett said   
Q2607                                   I said   
Q2608                               Brett said   
Q2609                                   I said   

                              menTexts  \
qId                                      
Q0                            [['us']]   
Q1                                [[]]   
Q2                        [[], ['We']]   
Q3     [[], ['you', 'you', 'Frances']]   
Q4                       [['Frances']]   
...                                ...   
Q2